# Import Package

In [1]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than

In [2]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
import pandas as pd
from autogluon.tabular import TabularPredictor
import warnings
warnings.filterwarnings('ignore')

# Read Data

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv').drop('id', axis=1)

In [4]:
test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv').drop('id', axis=1)

In [5]:
submission = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')

In [6]:
train.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


# Model Training

In [7]:
predictor = TabularPredictor(
    label='price',                       # Specify the target variable
    eval_metric='rmse',                 # Evaluation metric for regression
    problem_type='regression'            # Specify the type of problem
).fit(
    train,                               # The training dataset
    presets='best_quality',              # Use the best quality preset for training
    time_limit=3600* 1,                # Set a time limit of 1 hour for training
    verbosity=2,                         # Verbosity level for training output
    excluded_model_types=['KNN'],        # Exclude KNN model from training
    num_gpus=2 # 使用1个GPU
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240926_152051"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       30.08 GB / 31.36 GB (95.9%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets o

In [8]:
trained_models = predictor.get_model_names()
print("Trained models:")
for model in trained_models:
    print(model)

Trained models:
LightGBMXT_BAG_L1
LightGBM_BAG_L1
RandomForestMSE_BAG_L1
CatBoost_BAG_L1
WeightedEnsemble_L2
LightGBMXT_BAG_L2
LightGBM_BAG_L2
RandomForestMSE_BAG_L2
WeightedEnsemble_L3


# Predictions

In [9]:
predictions = predictor.predict(test)

print(predictions)

0         18760.792969
1         82778.734375
2         55734.781250
3         29128.720703
4         31690.824219
              ...     
125685    27879.300781
125686    52023.554688
125687    20284.613281
125688    15673.270508
125689    36662.671875
Name: price, Length: 125690, dtype: float32


In [10]:
print(submission.head(10))

       id      price
0  188533  43878.016
1  188534  43878.016
2  188535  43878.016
3  188536  43878.016
4  188537  43878.016
5  188538  43878.016
6  188539  43878.016
7  188540  43878.016
8  188541  43878.016
9  188542  43878.016


In [11]:
submission["price"] =  predictions
submission.to_csv('submission.csv',index=False)